<a href="https://colab.research.google.com/github/Launcha10/sports_entertaiment_topic_analysis/blob/kawamura/sports_randomforest.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [2]:
!apt install aptitude

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following package was automatically installed and is no longer required:
  libnvidia-common-430
Use 'apt autoremove' to remove it.
The following additional packages will be installed:
  aptitude-common libcgi-fast-perl libcgi-pm-perl libclass-accessor-perl
  libcwidget3v5 libencode-locale-perl libfcgi-perl libhtml-parser-perl
  libhtml-tagset-perl libhttp-date-perl libhttp-message-perl libio-html-perl
  libio-string-perl liblwp-mediatypes-perl libparse-debianchangelog-perl
  libsigc++-2.0-0v5 libsub-name-perl libtimedate-perl liburi-perl libxapian30
Suggested packages:
  aptitude-doc-en | aptitude-doc apt-xapian-index debtags tasksel
  libcwidget-dev libdata-dump-perl libhtml-template-perl libxml-simple-perl
  libwww-perl xapian-tools
The following NEW packages will be installed:
  aptitude aptitude-common libcgi-fast-perl libcgi-pm-perl
  libclass-accessor-perl libcwidget3v5 libencode-l

In [3]:
!aptitude install mecab libmecab-dev mecab-ipadic-utf8 git make curl xz-utils file -y

git is already installed at the requested version (1:2.17.1-1ubuntu0.5)
make is already installed at the requested version (4.1-9.1ubuntu1)
curl is already installed at the requested version (7.58.0-2ubuntu3.8)
xz-utils is already installed at the requested version (5.2.2-1.3)
git is already installed at the requested version (1:2.17.1-1ubuntu0.5)
make is already installed at the requested version (4.1-9.1ubuntu1)
curl is already installed at the requested version (7.58.0-2ubuntu3.8)
xz-utils is already installed at the requested version (5.2.2-1.3)
The following NEW packages will be installed:
  file libmagic-mgc{a} libmagic1{a} libmecab-dev libmecab2{a} mecab mecab-ipadic{a} mecab-ipadic-utf8 mecab-jumandic{a} mecab-jumandic-utf8{a} mecab-utils{a} 
The following packages will be REMOVED:
  libnvidia-common-430{u} 
0 packages upgraded, 11 newly installed, 1 to remove and 25 not upgraded.
Need to get 29.3 MB of archives. After unpacking 282 MB will be used.
Get: 1 http://archive.ubuntu

In [4]:
!pip install mecab-python3==0.7

     |████████████████████████████████| 51kB 3.9MB/s 
  Created wheel for mecab-python3: filename=mecab_python3-0.7-cp36-cp36m-linux_x86_64.whl size=155490 sha256=7c267ed2de95255c4d08364126acbe19a49d3c424525db52dff4f0482b603657
  Stored in directory: /root/.cache/pip/wheels/4c/07/3a/5f22ccc9f381f3bc01fa023202061cd1e0e9af855292f005dd
Successfully built mecab-python3


In [14]:
# -*- coding: utf-8 -*-
import os
import sys
import re
from gensim import corpora, matutils
import MeCab

DATA_DIR_PATH = '/content/drive/My Drive/sports_text/'
DICTIONARY_FILE_NAME = 'sportsdic.txt'
mecab = MeCab.Tagger('Owakati')


def get_class_id(file_name):
    '''
    ファイル名から、クラスIDを決定する。
    学習データを作るときに使っています。
    '''
    #print("file_nameだよ："+file_name)
    dir_list = get_dir_list()
    #print("dir_listだよ：")
    #print(dir_list)
    
    dir_name = next(filter(lambda x: x in file_name, dir_list),None)
    
    #print("dir_nameだよ：")
    #print(dir_name)
    if dir_name:
        return dir_list.index(dir_name)
    return None
def get_dir_list():
    '''
    ライブドアコーパスが./text/ の下にカテゴリ別にあるからそのカテゴリ一覧をとってるだけ
    '''
    tmp = os.listdir(DATA_DIR_PATH)
    #print(tmp)
    #print("----------------------")
    if tmp is None:
        return None
    return sorted([x for x in tmp if os.path.isdir(DATA_DIR_PATH + x)])


def get_file_content(file_path):
    '''
    1つの記事を読み込み
    '''
    with open(file_path, encoding='utf-8') as f:
        return ''.join(f.readlines()[1:])  # ライブドアコーパスが3行目から本文はじまってるから


def tokenize(text):
    '''
    とりあえず形態素解析して名詞だけ取り出す感じにしてる
    '''
    node = mecab.parseToNode(text)
    while node:
        if node.feature.split(',')[0] == '名詞':
            yield node.surface.lower()
        node = node.next


def check_stopwords(word):
    '''
    ストップワードだったらTrueを返す
    '''
    if re.search(r'^[0-9]+$', word):  # 数字だけ
        return True
    return False


def get_words(contents):
    '''
    記事群のdictについて、形態素解析して返す
    '''
    ret = []
    for k, content in contents.items():
        ret.append(get_words_main(content))
    return ret


def get_words_main(content):
    '''
    一つの記事を形態素解析して返す
    '''
    return [token for token in tokenize(content) if not check_stopwords(token)]


def filter_dictionary(dictionary):
    '''
    低頻度と高頻度のワードを除く感じで
    '''
    dictionary.filter_extremes(no_below=2, no_above=0.5)  # この数字はあとで変えるかも
    return dictionary


def get_contents():
    '''
    livedoorニュースのすべての記事をdictでまとめておく
    '''
    dir_list = get_dir_list()
    #print(dir_list)
    #print("-------------------------")

    if dir_list is None:
        return None

    ret = {}
    for dir_name in dir_list:
        file_list = os.listdir(DATA_DIR_PATH + dir_name)
        print(file_list)
        #print("--------------------")

        if file_list is None:
            continue
        for file_name in file_list:
            #if dir_name in file_name:  # LICENSE.txt とかを除くためです。。
                ret[file_name] = get_file_content(DATA_DIR_PATH + dir_name + '/' + file_name)
        #print(len(ret))    
        #print("-------------------")

    return ret


def get_vector(dictionary, content):
    '''
    ある記事の特徴語カウント
    '''
    tmp = dictionary.doc2bow(get_words_main(content))
    dense = list(matutils.corpus2dense([tmp], num_terms=len(dictionary)).T[0])
    return dense


def get_dictionary(create_flg=False, file_name=DICTIONARY_FILE_NAME):
    '''
    辞書を作る
    '''
    if create_flg or not os.path.exists(file_name):
        # データ読み込み
        contents = get_contents()
        
        # 形態素解析して名詞だけ取り出す
        words = get_words(contents)
        #print("retだよ")
        #print(words)
        # 辞書作成、そのあとフィルタかける
        dictionary = filter_dictionary(corpora.Dictionary(words))
        #print("dicだよ")
        #print(dictionary)
        # 保存しておく
        if file_name is None:
            sys.exit()
        dictionary.save_as_text(file_name)

    else:
        # 通常はファイルから読み込むだけにする
        dictionary = corpora.Dictionary.load_from_text(file_name)

    return dictionary


if __name__ == '__main__':
    get_dictionary(create_flg=True)
    




['baseball-text2.txt', 'baseball-text3.txt', 'baseball-text4.txt', 'baseball-text5.txt', 'baseball-text6.txt', 'baseball-text7.txt', 'baseball-text8.txt', 'baseball-text9.txt', 'baseball-text10.txt', 'baseball-text11.txt', 'baseball-text12.txt', 'baseball-text13.txt', 'baseball-text14.txt', 'baseball-text15.txt', 'baseball-text16.txt', 'baseball-text17.txt', 'baseball-text18.txt', 'baseball-text19.txt', 'baseball-text20.txt', 'baseball-text21.txt', 'baseball-text22.txt', 'baseball-text23.txt', 'baseball-text24.txt', 'baseball-text25.txt', 'baseball-text26.txt', 'baseball-text27.txt', 'baseball-text28.txt', 'baseball-text29.txt', 'baseball-text30.txt', 'baseball-text31.txt', 'baseball-text32.txt', 'baseball-text33.txt', 'baseball-text34.txt', 'baseball-text35.txt', 'baseball-text36.txt', 'baseball-text37.txt', 'baseball-text38.txt', 'baseball-text39.txt', 'baseball-text40.txt', 'baseball-text41.txt', 'baseball-text42.txt', 'baseball-text43.txt', 'baseball-text44.txt', 'baseball-text45.t

In [6]:
#scikit-learnのインストール
pip install scikit-learn

In [7]:
import sklearn

print(sklearn.__version__)

0.22.1


In [15]:
# -*- coding: utf-8 -*-
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report
#import corpus
# 不要な警告を非表示にする
import warnings
warnings.filterwarnings('ignore')


def main():
    # 辞書の読み込み
    dictionary = get_dictionary(create_flg=False)
    print(dictionary)
    # 記事の読み込み
    contents = get_contents()

    # 特徴抽出
    data_train = []
    label_train = []
    for file_name, content in contents.items():

        data_train.append(get_vector(dictionary, content))
        a = label_train.append(get_class_id(file_name))
    print(a)
    print(data_train)
    print(label_train)

    # 分類器
    estimator = RandomForestClassifier()

    # 学習
    estimator.fit(data_train, label_train)

    # 学習したデータを予測にかけてみる（ズルなので正答率高くないとおかしい）
    print("==== 学習データと予測データが一緒の場合")
    print(estimator.score(data_train, label_train))

    # 学習データと試験データに分けてみる
    data_train_s, data_test_s, label_train_s, label_test_s = train_test_split(data_train, label_train, test_size=0.5)

    # 分類器をもう一度定義
    estimator2 = RandomForestClassifier()

    # 学習
    estimator2.fit(data_train_s, label_train_s)
    print("==== 学習データと予測データが違う場合")
    print(estimator2.score(data_test_s, label_test_s))
    print(data_train_s)
    print(label_train_s)

    
if __name__ == '__main__':
    main()

Dictionary(10988 unique tokens: ['―、―、―、―', '―、―、―、―、―', '、', 'あいさつ', 'あいつ']...)
['baseball-text2.txt', 'baseball-text3.txt', 'baseball-text4.txt', 'baseball-text5.txt', 'baseball-text6.txt', 'baseball-text7.txt', 'baseball-text8.txt', 'baseball-text9.txt', 'baseball-text10.txt', 'baseball-text11.txt', 'baseball-text12.txt', 'baseball-text13.txt', 'baseball-text14.txt', 'baseball-text15.txt', 'baseball-text16.txt', 'baseball-text17.txt', 'baseball-text18.txt', 'baseball-text19.txt', 'baseball-text20.txt', 'baseball-text21.txt', 'baseball-text22.txt', 'baseball-text23.txt', 'baseball-text24.txt', 'baseball-text25.txt', 'baseball-text26.txt', 'baseball-text27.txt', 'baseball-text28.txt', 'baseball-text29.txt', 'baseball-text30.txt', 'baseball-text31.txt', 'baseball-text32.txt', 'baseball-text33.txt', 'baseball-text34.txt', 'baseball-text35.txt', 'baseball-text36.txt', 'baseball-text37.txt', 'baseball-text38.txt', 'baseball-text39.txt', 'baseball-text40.txt', 'baseball-text41.txt', 'baseb

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



==== 学習データと予測データが一緒の場合
0.9920792079207921
==== 学習データと予測データが違う場合
0.9881422924901185


IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [0]:
# -*- coding: utf-8 -*-
import os
import sys
import re
from gensim import corpora, matutils
import MeCab

DATA_DIR_PATH = '/content/drive/My Drive/text/'
DICTIONARY_FILE_NAME = 'livedoordic.txt'
mecab = MeCab.Tagger('Owakati')


def get_class_id(file_name):
    '''
    ファイル名から、クラスIDを決定する。
    学習データを作るときに使っています。
    '''
    dir_list = get_dir_list()
    dir_name = next(filter(lambda x: x in file_name, dir_list), None)
    if dir_name:
        return dir_list.index(dir_name)
    return None
def get_dir_list():
    '''
    ライブドアコーパスが./text/ の下にカテゴリ別にあるからそのカテゴリ一覧をとってるだけ
    '''
    tmp = os.listdir(DATA_DIR_PATH)
    if tmp is None:
        return None
    return sorted([x for x in tmp if os.path.isdir(DATA_DIR_PATH + x)])


def get_file_content(file_path):
    '''
    1つの記事を読み込み
    '''
    with open(file_path, encoding='utf-8') as f:
        return ''.join(f.readlines()[2:])  # ライブドアコーパスが3行目から本文はじまってるから


def tokenize(text):
    '''
    とりあえず形態素解析して名詞だけ取り出す感じにしてる
    '''
    node = mecab.parseToNode(text)
    while node:
        if node.feature.split(',')[0] == '名詞':
            yield node.surface.lower()
        node = node.next


def check_stopwords(word):
    '''
    ストップワードだったらTrueを返す
    '''
    if re.search(r'^[0-9]+$', word):  # 数字だけ
        return True
    return False


def get_words(contents):
    '''
    記事群のdictについて、形態素解析して返す
    '''
    ret = []
    for k, content in contents.items():
        ret.append(get_words_main(content))
    return ret


def get_words_main(content):
    '''
    一つの記事を形態素解析して返す
    '''
    return [token for token in tokenize(content) if not check_stopwords(token)]


def filter_dictionary(dictionary):
    '''
    低頻度と高頻度のワードを除く感じで
    '''
    dictionary.filter_extremes(no_below=20, no_above=0.3)  # この数字はあとで変えるかも
    return dictionary


def get_contents():
    '''
    livedoorニュースのすべての記事をdictでまとめておく
    '''
    dir_list = get_dir_list()

    if dir_list is None:
        return None

    ret = {}
    for dir_name in dir_list:
        file_list = os.listdir(DATA_DIR_PATH + dir_name)

        if file_list is None:
            continue
        for file_name in file_list:
            if dir_name in file_name:  # LICENSE.txt とかを除くためです。。
                ret[file_name] = get_file_content(DATA_DIR_PATH + dir_name + '/' + file_name)

    return ret


def get_vector(dictionary, content):
    '''
    ある記事の特徴語カウント
    '''
    tmp = dictionary.doc2bow(get_words_main(content))
    dense = list(matutils.corpus2dense([tmp], num_terms=len(dictionary)).T[0])
    return dense


def get_dictionary(create_flg=False, file_name=DICTIONARY_FILE_NAME):
    '''
    辞書を作る
    '''
    if create_flg or not os.path.exists(file_name):
        # データ読み込み
        contents = get_contents()
        # 形態素解析して名詞だけ取り出す
        words = get_words(contents)
        # 辞書作成、そのあとフィルタかける
        dictionary = filter_dictionary(corpora.Dictionary(words))
        # 保存しておく
        if file_name is None:
            sys.exit()
        dictionary.save_as_text(file_name)

    else:
        # 通常はファイルから読み込むだけにする
        dictionary = corpora.Dictionary.load_from_text(file_name)

    return dictionary


if __name__ == '__main__':
    get_dictionary(create_flg=True)


